# Кольцо вычетов по модулю

In [1]:
class DeductionClassException(Exception):
    pass


class DeductionClass:
    '''Класс вычетов a по модулю m.'''
    def __init__(self,a,m):
        if not(isinstance(m,int) and m>1):
            raise DeductionClassException("Модуль должен быть натуральным числом, большим единицы.")
        elif not(isinstance(a,int)):
            raise DeductionClassException("Класс вычетов должен быть целым числом.")
        else:
            self.m=m
            self.a=a%m
    
    def __str__(self):
        return "["+str(self.a)+"]"+str(self.m)
    
    def __eq__(self,other):
        return (self.a==other.a)and(self.m==other.m)
    
    def __add__(self,other):
        if not(isinstance(other,DeductionClass)):
            raise DeductionClassException("Класс вычетов можно складывать только с классом вычетов.")
        else:
            if self.m==other.m:
                return DeductionClass(self.a+other.a,self.m)
            else:
                raise DeductionClassException("Складывать можно только классы вычетов по одному модулю.")
    
    def __radd__(self,other):
        return self+other
    
    def __neg__(self):
        return DeductionClass(-self.a,self.m)
    
    def __sub__(self,other):
        return self+(-other)
    
    def __abs__(self):
        return self
    
    def __mul__(self,other):
        if not(isinstance(other,DeductionClass)):
            raise DeductionClassException("Класс вычетов можно умножать только на класс вычетов.")
        else:
            if self.m==other.m:
                return DeductionClass(self.a*other.a,self.m)
            else:
                raise DeductionClassException("Перемножать можно только классы вычетов по одному модулю.")
    
    def __rmul__(self,other):
        return self*other

# Поле вычетов по простому модулю

In [2]:
def number_is_prime(n):
    i=2
    e=n**0.5
    while i<=e:
        if not(n%i):
            return False
        i+=1
    return True

def extended_Euclid(a,b):
    if a==0:
        return (b,0,1)
    else:
        g,x,y=extended_Euclid(b%a,a)
        return (g,y-(b//a)*x,x)


class DeductionClassPrime(DeductionClass):
    '''Класс вычетов a по простому модулю m.'''
    def __init__(self,a,m):
        if number_is_prime(m):
            super(DeductionClassPrime,self).__init__(a,m)
        else:
            raise DeductionClassException("Модуль должен быть простым числом.")
    
    def reverse(self):
        if(self.a!=0):
            t=extended_Euclid(self.a,self.m)
            return DeductionClassPrime(t[1],self.m)
        else:
            raise ZeroDivisionError("Не существует обратного к нулю.")
    
    def __truediv__(self,other):
        return self*other.reverse()
    
    @staticmethod
    def to_prime(db):
        if number_is_prime(db.m):
            return DeductionClassPrime(db.a,db.m)
        else:
            raise DeductionClassException("Модуль должен быть простым числом.")

# Кольцо многочленов
Перед использованием класса нужно обязательно определить для него default -- функцию, которая возвращает нуль кольца, над которым определен многочлен, и epsilon -- наибольший ненулевой элемент кольца. Если в кольце не определено отношение порядка (<=), многочлен может работать неверно. Возможно, имеет смысл определить его как в примере.

In [15]:
from collections import defaultdict

def maxel(l):
    r=l[0]
    for e in l[1:]:
        if r<e:
            r=e
    return r

class PolynomialException(Exception):
    pass

class PolynomialRingItem:
    '''Многочлен над кольцом.'''
    
    @staticmethod
    def set_default(f):
        PolynomialRingItem.default=f
        PolynomialRingItem.coefficients_type=type(f())
    
    @staticmethod
    def set_epsilon(e):
        PolynomialRingItem.epsilon=e
    
    def __init__(self,d):
        self.d=defaultdict(PolynomialRingItem.default)
        for k,v in d.items():
            if (not isinstance(k,int))or k<0:
                raise PolynomialException("Степень x должна быть натуральным числом или нулем.")
            if not isinstance(v,PolynomialRingItem.coefficients_type):
                raise PolynomialException("Многочлен не может быть определен над разнородными объектами.")
            else:
                self.d[k]=v
        self.cleanup()
    
    def __str__(self):
        keys=list(self.d.keys())
        keys.sort(reverse=True)
        s=""
        for k in keys:
            s+=self.d[k].__str__()+"*x^"+k.__str__()+"+"
        return s[0:-1]
    
    def cleanup(self):
        d=[]
        for k in self.d.keys():
            if self.d[k].__abs__()<=PolynomialRingItem.epsilon:
                d.append(k)
        for k in d:
            del self.d[k]
        return self
    
    def deg(self):
        self.cleanup()
        return maxel(list(self.d.keys()))
    
    def __eq__(self,other):
        self.cleanup()
        other.cleanup()
        if(self.deg()!=other.deg()):
            return False
        for k in self.d.keys():
            if(self.d[k]!=other.d[k]):
                return False
        return True
    
    def __add__(self,other):
        if not(isinstance(other,PolynomialRingItem)):
            raise PolynomialException("Многочлен можно складывать только с другим многочленом.")
        else:
            for k,v in self.d.items():
                other.d[k]+=v
        return other.cleanup()
    
    def __radd__(self,other):
        return self+other
    
    def __neg__(self):
        s=self.d.copy()
        for k,v in s.items():
            s[k]=-v
        return PolynomialRingItem(s)
    
    def __sub__(self,other):
        return self+(-other)
    
    def __mul__(self,other):
        if not(isinstance(other,PolynomialRingItem)):
            raise PolynomialException("Многочлен можно складывать только с другим многочленом.")
        else:
            deg=self.deg()+other.deg()
            d={}
            for i in range(0,deg+1):
                d[i]=PolynomialRingItem.default()
                for j in range(0,i+1):
                    d[i]+=self.d[j]*other.d[i-j]
            return PolynomialRingItem(d).cleanup()
    
    def __rmul__(self,other):
        return self*other

*Пример:*

In [16]:
# В кольце вычетов по модулю 4 нет отношения порядка, однако, задав его следующим образом, мы заставим многочлен чистить нули
def f(self,other):
    if(self==other):
        return True
    else:
        return False
DeductionClass.__le__=f
# Определяем ноль и epsilon многочлена
PolynomialRingItem.set_default(lambda: DeductionClass(0,4))
PolynomialRingItem.set_epsilon(DeductionClass(0,4))
# Как мы видим, все работает верно (в кольце вычетов по модулю 4 1+1=0)
d={0:DeductionClass(1,4),1:DeductionClass(2,4)}
p=PolynomialRingItem(d)
print(p)
print(p*p)

[2]4*x^1+[1]4*x^0
[1]4*x^0
